# k-NN and Facial Recognition

### 1. Script for adding a new face

#### Load libraries

In [2]:
import os
import numpy as np
import cv2
import pickle

###### Use case of imported libraries:

 **numpy** library because it is used to represent images as arrays, perform operations on these arrays, and manipulate feature vectors extracted from face images. 

**OS** library because it provides a way to interact with the operating system in a platform-independent manner. 

**cv2** library because it is used to perform tasks such as face detection and alignment. 

**pickle** library because it is used to save and load trained models.

In [8]:
import cv2
import numpy as np
import pickle
import os

# Initialize variables
face_data = []
i = 0

# Video capture object to access the webcam
camera = cv2.VideoCapture(0)  # Use 1 for external camera

# Create Haar cascade object to detect faces in the frame
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

name = input('Enter your name --> ')
ret = True  # Start infinite loop

# The loop for facial recognition using KNN
while ret:
    ret, frame = camera.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert the image to grayscale

        face_coordinates = face_cascade.detectMultiScale(gray, 1.3, 4)  # Detect face coordinates

        for (x, y, w, h) in face_coordinates:
            faces = frame[y:y+h, x:x+w, :]
            resized_faces = cv2.resize(faces, (50, 50))  # Crop and resize the face region

            if i % 10 == 0 and len(face_data) < 10:
                face_data.append(resized_faces)  # Add resized face to the list every 10 frames
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        i += 1

        cv2.imshow('frames', frame)

        if cv2.waitKey(1) == 27 or len(face_data) >= 10:  # Break on 'ESC' key or if enough data is collected
            break
    else:
        print('Error in capturing video')
        break

cv2.destroyAllWindows()
camera.release()

face_data = np.asarray(face_data)
face_data = face_data.reshape(10, -1)

save_path = r'C:\Users\0022491\Desktop\IIE VARSITY COLLEGE\2024\ice 1\kNN_28042024'

# Save names
names_file = os.path.join(save_path, 'names.pkl')
if not os.path.exists(names_file):
    names = [name] * 10
    with open(names_file, 'wb') as file:
        pickle.dump(names, file)
else:
    with open(names_file, 'rb') as file:
        names = pickle.load(file)
    names += [name] * 10
    with open(names_file, 'wb') as file:
        pickle.dump(names, file)

# Save faces
faces_file = os.path.join(save_path, 'faces.pkl')
if not os.path.exists(faces_file):
    with open(faces_file, 'wb') as file:
        pickle.dump(face_data, file)
else:
    with open(faces_file, 'rb') as file:
        faces = pickle.load(file)
    faces = np.append(faces, face_data, axis=0)
    with open(faces_file, 'wb') as file:
        pickle.dump(faces, file)

    


Enter your name --> R


### Script for live face recognition using knn

In [10]:
import pickle
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import os

# Load the face data and names
save_path = r'C:\Users\0022491\Desktop\IIE VARSITY COLLEGE\2024\ice 1\kNN_28042024'

with open(os.path.join(save_path, 'faces.pkl'), 'rb') as file:
    faces = pickle.load(file)

with open(os.path.join(save_path, 'names.pkl'), 'rb') as file:
    names = pickle.load(file)

# Create and train the KNN model
knn = KNeighborsClassifier(n_neighbors=5)  # You can tune the number of neighbors
knn.fit(faces, names)

# Save the trained KNN model
with open(os.path.join(save_path, 'knn_model.pkl'), 'wb') as file:
    pickle.dump(knn, file)

In [11]:
import cv2
import numpy as np
import pickle
import os

# Load the trained KNN model
save_path = r'C:\Users\0022491\Desktop\IIE VARSITY COLLEGE\2024\ice 1\kNN_28042024'

with open(os.path.join(save_path, 'knn_model.pkl'), 'rb') as file:
    knn = pickle.load(file)

# Initialize the video capture
camera = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = camera.read()
    if not ret:
        print("Error in capturing video")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_coordinates = face_cascade.detectMultiScale(gray, 1.3, 4)
    
    for (x, y, w, h) in face_coordinates:
        faces = frame[y:y+h, x:x+w, :]
        resized_faces = cv2.resize(faces, (50, 50)).reshape(1, -1)
        
        # Predict the name of the person using KNN
        name = knn.predict(resized_faces)[0]
        
        # Display the name on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    cv2.imshow('Face Recognition', frame)
    
    if cv2.waitKey(1) == 27:  # Break on 'ESC' key
        break

cv2.destroyAllWindows()
camera.release()


###  Script for Static Facial recognition with K-NN

In [ ]:
import cv2
import numpy as np
import pickle
import os

# Load the pre-trained KNN model
save_path = r'C:\Users\0022491\Desktop\IIE VARSITY COLLEGE\2024\ice 1\kNN_28042024'

with open(os.path.join(save_path, 'knn_model.pkl'), 'rb') as file:
    knn = pickle.load(file)

# Load the static image
image_path = r'C:\Users\0022491\Desktop\IIE VARSITY COLLEGE\2024\ice 1\kNN_28042024\nicholas_cage\cage.jpeg' 
image = cv2.imread(image_path)

if image is None:
    print("Error: Unable to load image")
else:
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Initialize the Haar Cascade face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Detect faces in the image
    face_coordinates = face_cascade.detectMultiScale(gray, 1.3, 4)
    
    if len(face_coordinates) == 0:
        print("No faces detected")
    else:
        for (x, y, w, h) in face_coordinates:
            faces = image[y:y+h, x:x+w, :]
            resized_faces = cv2.resize(faces, (50, 50)).reshape(1, -1)
            
            # Predict the name of the person using KNN
            name = knn.predict(resized_faces)[0]
            
            # Draw a rectangle around the face
            cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
            # Display the name on the image
            cv2.putText(image, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    # Show the image with detected faces
    cv2.imshow('Face Recognition', image)
    
    # Wait until a key is pressed and then close the image window
    cv2.waitKey(0)
    cv2.destroyAllWindows()
